# Data analysis with python

In [ ]:
# ------ 0) Wgranie potrzebnych ramek danych:
import pandas as pd
import numpy as np
import os, os.path
import sqlite3

Tags = pd.read_csv("Tags.csv")
Badges = pd.read_csv("Badges.csv")
Posts = pd.read_csv("Posts.csv")
Users = pd.read_csv("Users.csv")

baza = os.path.join('przyklad.db')
if os.path.isfile(baza): 
	os.remove(baza) 

conn = sqlite3.connect(baza)
Badges.to_sql("Badges", conn) 
Posts.to_sql("Posts", conn)
Tags.to_sql("Tags", conn)
Users.to_sql("Users", conn)


# Zadanie 1

In [ ]:
# Zadanie ------ 1) rozwiązanie sql:

sqldf1 = pd.read_sql_query("""
SELECT Count, TagName
FROM Tags
WHERE Count > 1000
ORDER BY Count DESC
""", conn)


In [4]:
# Zadanie ------ 1) rozwiązanie pandas:

edited_Tags = Tags[["TagName", "Count"]] # Wybieramy kolumny na których będziemy operować
edited_Tags = edited_Tags.loc[edited_Tags["Count"] > 1000] # Wybieramy tagi, których jest więcej niż 1000
edited_Tags = edited_Tags.sort_values(by=['Count'], ascending = False) # układamy zliczenia tagów malejąco
pandas_result1 = edited_Tags[["Count","TagName"]].reset_index(drop=True) # usuwamy kolumny indeksów


In [ ]:
# Zadanie ------ 1) sprawdzenie wyników:

print(type(pandas_result1)) # podaje wynik DataFrame
print(sqldf1.equals(pandas_result1)) # podaje wynik True


# Zadanie 2

In [ ]:
# Zadanie ------ 2) rozwiązanie sql:

sqldf2 = pd.read_sql_query("""
SELECT Location, COUNT(*) AS Count
  FROM (
  SELECT Posts.OwnerUserId, Users.Id, Users.Location
  FROM Users
  JOIN Posts ON Users.Id = Posts.OwnerUserId
  )
  WHERE Location NOT IN ('')
  GROUP BY Location
  ORDER BY Count DESC
  LIMIT 10
""", conn)


In [ ]:
# Zadanie ------ 2) rozwiązanie pandas:

edited_Users = Users[["Id","Location"]] # Wybieramy potrzebne kolumny
edited_Users = edited_Users.loc[edited_Users["Location"]!=""] # eliminujemy puste lokalizacje
edited_Posts = Posts["OwnerUserId"] # wybieramy potrzebną kolumnę z Posts
edited_Posts = edited_Posts.to_frame() # Zmieniamy z Series na dataFrame
edited_Posts = edited_Posts.rename({"OwnerUserId":"Id"},axis=1) # Zmieniamy nazwę kolumny na "Id"
merge = pd.merge(edited_Users,edited_Posts, on="Id") # Dokonujemy merge wedle Id
pandas_result2 = merge.groupby(["Location"])["Location"].count() # grupujemy po lokacjach i zliczamy
pandas_result2 = pandas_result2.to_frame() # Zmieniamy z Series na dataFrame
pandas_result2.columns = ["Count"] # Zmieniamy nazwę kolumny wyliczonych lokacji na Count
pandas_result2 = pandas_result2.sort_values(["Count"], ascending = False).head(10) # sortujemy dane i podajemy 10 wyników
pandas_result2['Location'] = pandas_result2.index # Zmieniamy kolumnę z indeksów na Location 
pandas_result2.index=range(0, len(pandas_result2)) # Tworzymy kolumnę indeksów
pandas_result2 = pd.DataFrame(pandas_result2,columns=["Location","Count"]) # Zamieniamy kolumny miejscami aby uzyskać wynik


In [ ]:
# Zadanie ------ 2) sprawdzenie wyników:

print(type(pandas_result2)) # Zwraca DataFrame
print(sqldf2.equals(pandas_result2)) # Zwraca True


# Zadanie 3

In [ ]:
# Zadanie ------ 3) rozwiązanie sql:

sqldf3 = pd.read_sql_query("""
SELECT Year, SUM(Number) AS TotalNumber
  FROM (
  SELECT
  Name,
  COUNT(*) AS Number,
  STRFTIME('%Y', Badges.Date) AS Year
  FROM Badges
  WHERE Class = 1
  GROUP BY Name, Year
  )
  GROUP BY Year
  ORDER BY TotalNumber
""", conn)

In [ ]:
# Zadanie ------ 3) rozwiązanie pandas:

edited_Badges = Badges.loc[Badges["Class"]==1][["Name","Date"]] # Wybieramy dane tylko z pierwszą klasą
date = edited_Badges["Date"] # Tworzymy wartość date do modyfikowania daty
Year = pd.DatetimeIndex(date).year  # Zmieniamy date aby został tylko rok
edited_Badges["Date"] = Year   # Podmieniamy date na zmodyfikowaną
pandas_result3 = edited_Badges.groupby(["Date"])["Date"].count() # Grupujemy i zliczamy po Date
pandas_result3 = pandas_result3.to_frame() # Zmieniamy z Series na DataFrame
pandas_result3.columns=["TotalNumber"] # Zmieniamy nazwe kolumny na TotalNumber
pandas_result3['Date'] = pandas_result3.index.astype(str) # Zmieniamy wartości dat na stringi i przenosimy do kolumny Date
pandas_result3.index=range(0, len(pandas_result3)) # Tworzymy kolumnę indeksów
pandas_result3 = pd.DataFrame(pandas_result3,columns=["Date","TotalNumber"]) # Zamieniamy kolumny do ostatecznego wyniku
pandas_result3 = pandas_result3.sort_values(["TotalNumber"], ascending = True).reset_index(drop=True) # Resetujemy kolumne indeksów i sortujemy 
pandas_result3.columns=["Year","TotalNumber"] # Zmieniamy nazwy kolumn aby uzyskać wynik


In [ ]:
# Zadanie ------ 3) sprawdzenie wyników:

print(type(pandas_result3)) # Zwraca DataFrame
print(sqldf3.equals(pandas_result3)) # Zwraca True


# Zadanie 4

In [ ]:
# Zadanie ------ 4) rozwiązanie sql:

sqldf4 = pd.read_sql_query("""
SELECT
  Users.AccountId,
  Users.DisplayName,
  Users.Location,
  AVG(PostAuth.AnswersCount) as AverageAnswersCount
  FROM
  (
  SELECT
  AnsCount.AnswersCount,
  Posts.Id,
  Posts.OwnerUserId
  FROM (
  SELECT Posts.ParentId, COUNT(*) AS AnswersCount
  FROM Posts
  WHERE Posts.PostTypeId = 2
  GROUP BY Posts.ParentId
  ) AS AnsCount
  JOIN Posts ON Posts.Id = AnsCount.ParentId
  ) AS PostAuth
  JOIN Users ON Users.AccountId=PostAuth.OwnerUserId
  GROUP BY OwnerUserId
  ORDER BY AverageAnswersCount DESC, AccountId ASC
  LIMIT 10
""", conn)


In [ ]:
# Zadanie ------ 4) rozwiązanie pandas:

edited_Posts = Posts.loc[Posts["PostTypeId"]==2][["ParentId"]] # Filtrujemy po PostTypeId równych 2
Id = Posts[["Id","OwnerUserId"]] # Zmieniamy nazwy kolumny
edited_User = Users[["AccountId","Location","DisplayName"]] # Wybieramy potrzebne kolumny
edited_User.columns = ["OwnerUserId","Location","DisplayName"] # Zmieniamy nazwy kolumn
AnsCount = edited_Posts.groupby(["ParentId"])["ParentId"].count() # Grupujemy i zliczamy po ParentId
AnsCount = AnsCount.to_frame() # Przekształcamy z Series do frame
AnsCount.columns=["TotalNumber"] # Zmieniamy nazwe kolumny w AnsCount
AnsCount['ParentId'] = AnsCount.index.astype(int) # zapisujemy kolumne ParentId jako int
AnsCount.index=range(0, len(AnsCount)) # Dodajemy kolumne indeksu
AnsCount = AnsCount.rename(columns={"ParentId":"Id"})  # Zmieniamy nazwe kolumny na Id
PostAuth = Id.merge(AnsCount,on="Id") # Łączymy wyniki po Id użytkownika
merge = edited_User.merge(PostAuth,on="OwnerUserId") # Łączymy wyniki z linijki wyżej i users po OwnerUserId
merge = merge.dropna(subset=["OwnerUserId"]) # Usuwamy nieistniejące OwnerUserId
average = merge[["OwnerUserId","TotalNumber"]].sort_values(["OwnerUserId"],ascending=True) # tworzymy zmienną do liczenia średniej
mean = average.groupby(["OwnerUserId"]).mean()
mean['OwnerUserId'] = mean.index # zapisujemy kolumne OwnerUser jako int
mean.index=range(0, len(mean)) # Tworzenie kolumny indeksów
pandas_result4 = edited_User.merge(mean,on="OwnerUserId") # merge po OwnerUser
pandas_result4 = pandas_result4.sort_values(["TotalNumber","OwnerUserId"],ascending=[False,True]).head(10) # Posortowanie i ustawienie 10 wyników
pandas_result4.columns = ["AccountId","Location","DisplayName","AverageAnswersCount"] # Zmiana nazw kolumn
pandas_result4 = pandas_result4[["AccountId","DisplayName","Location","AverageAnswersCount"]].reset_index().drop(columns="index") # Zamiana kolumn


In [ ]:
# Zadanie ------ 4) sprawdzenie wyników:

print(type(pandas_result4)) # Zwraca DataFrame
print(sqldf4.equals(pandas_result4)) # Zwraca True
